In [ ]:
from pasteur.kedro.ipython import * # type: ignore
register_kedro() # type: ignore

%load_ext autoreload
%autoreload 2

INFO     Kedro project Pasteur                                                                                           __init__.py:77
INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines'                                         __init__.py:78
INFO     Registered line magic 'run_viz'                                                                                 __init__.py:84


In [ ]:
# Use the first partition of texas_charges with the fit transformer
attrs = catalog.load('mimic_billion.trn.table')['idx'].get_attributes()
wrk = catalog.load('mimic_billion.wrk.idx_table')

import pandas as pd
import numpy as np
table = wrk.sample()
table.shape

(14025507, 16)


In [ ]:
Ns = [
    20_000,
    500_000,
    1_000_000,
    2_000_000,
    5_000_000,
    10_000_000
]

N_dom = 5_000_000

In [ ]:
from pasteur.hierarchy import rebalance_attributes
from pasteur.marginal import MarginalOracle
from pasteur.utils import LazyDataset, LazyPartition

m = MarginalOracle(attrs, LazyDataset(LazyPartition(lambda: table, lambda: table.shape)))
counts = m.get_counts()
m.close()

new_attrs = rebalance_attributes(counts, attrs, fixed=[4, 9, 18, 32], u=4)

WARNING  Launching a process pool implicitly. Use `init_pool()` to explicitly control pool creation.                    progress.py:227
INFO     Processed 16 marginals.                                                                                        __init__.py:410
WARNING  Rebalancing columns without using Differential Privacy (e_p=inf)                                              hierarchy.py:442


In [ ]:
for name, attr in new_attrs.items():
    for val_name, val in attr.vals.items():
        print(f"{val_name:>25s} | {', '.join(f'{h}:{val.get_domain(h):3d}' for h in range(val.height))}")


              intime_year | 0: 32, 1: 18, 2:  9, 3:  4
              intime_week | 0: 52, 1: 32, 2: 18, 3:  9, 4:  4
               intime_day | 0:  7, 1:  4
              intime_time | 0: 24, 1: 18, 2:  9, 3:  4
             outtime_week | 0:  8, 1:  4
              outtime_day | 0:  7, 1:  4
             outtime_time | 0: 24, 1: 18, 2:  9, 3:  4
           charttime_week | 0: 12, 1:  9, 2:  4
            charttime_day | 0:  7, 1:  4
           charttime_time | 0: 24, 1: 18, 2:  9, 3:  4
                 valuenum | 0: 33, 1: 32, 2: 18, 3:  9, 4:  4, 5:  2
                 valueuom | 0: 56, 1: 32, 2: 18, 3:  9, 4:  4, 5:  3
                  warning | 0:  2
           first_careunit | 0: 10, 1:  9, 2:  4, 3:  2
            last_careunit | 0: 10, 1:  9, 2:  4, 3:  2
                   gender | 0:  2


In [ ]:
from pasteur.marginal import AttrSelector, AttrSelectors, expand_table

In [ ]:
m2_20 = {'revenue_code': 16, 'hcpcs_qualifier': 2, 'hcpcs_procedure_code': 23, 'modifier_1': 10, 'modifier_2': 5, 'unit_measurement_code': 2, 'units_of_service': 6, 'unit_rate': 6, 'chrgs_line_item': 5, 'chrgs_non_cov': 5}
m2_24 = {'revenue_code': 16, 'hcpcs_qualifier': 2, 'hcpcs_procedure_code': 20, 'modifier_1': 8, 'modifier_2': 5, 'unit_measurement_code': 1, 'units_of_service': 5, 'unit_rate': 5, 'chrgs_line_item': 4, 'chrgs_non_cov': 4}
m2_28 = {'revenue_code': 12, 'hcpcs_qualifier': 2, 'hcpcs_procedure_code': 18, 'modifier_1': 6, 'modifier_2': 5, 'unit_measurement_code': 1, 'units_of_service': 5, 'unit_rate': 5, 'chrgs_line_item': 4, 'chrgs_non_cov': 4}
m2_32 = {'revenue_code': 9, 'hcpcs_qualifier': 2, 'hcpcs_procedure_code': 16, 'modifier_1': 4, 'modifier_2': 5, 'unit_measurement_code': 1, 'units_of_service': 4, 'unit_rate': 4, 'chrgs_line_item': 4, 'chrgs_non_cov': 4}

mar = {
    "gender": AttrSelector("gender", 0, {"gender": 0}),
    "warning": AttrSelector("warning", 0, {"warning": 0}),
    "intime": AttrSelector("intime", 0, {"intime_day": 1}),
    "outtime": AttrSelector("outtime", 0, {"outtime_day": 1}),
    "charttime": AttrSelector(
        "charttime", 0, {"charttime_day": 0, "charttime_time": 2}
    ),
}

m2_8 = {
    "gender": AttrSelector("gender", 0, {"gender": 0}),
    "warning": AttrSelector("warning", 0, {"warning": 0}),
    "first_careunit": AttrSelector("first_careunit", 0, {"first_careunit": 0}),
    "last_careunit": AttrSelector("last_careunit", 0, {"last_careunit": 2}),
}
m2_12 = {
    "gender": AttrSelector("gender", 0, {"gender": 0}),
    "warning": AttrSelector("warning", 0, {"warning": 0}),
    "first_careunit": AttrSelector("first_careunit", 0, {"first_careunit": 1}),
    "last_careunit": AttrSelector("last_careunit", 0, {"last_careunit": 1}),
    "intime": AttrSelector("intime", 0, {"intime_day": 1}),
    "outtime": AttrSelector("outtime", 0, {"outtime_day": 1}),
}
m2_16 = {
    "gender": AttrSelector("gender", 0, {"gender": 0}),
    "warning": AttrSelector("warning", 0, {"warning": 0}),
    "intime": AttrSelector("intime", 0, {"intime_day": 1}),
    "outtime": AttrSelector("outtime", 0, {"outtime_day": 1}),
    "charttime": AttrSelector(
        "charttime", 0, {"charttime_day": 0, "charttime_time": 2}
    ),
    "first_careunit": AttrSelector("first_careunit", 0, {"first_careunit": 2}),
    "valueuom": AttrSelector("valueuom", 0, {"valueuom": 5}),
}
m2_20 = {
    "gender": AttrSelector("gender", 0, {"gender": 0}),
    "warning": AttrSelector("warning", 0, {"warning": 0}),
    "first_careunit": AttrSelector("first_careunit", 0, {"first_careunit": 0}),
    "last_careunit": AttrSelector("last_careunit", 0, {"last_careunit": 0}),
    "intime": AttrSelector("intime", 0, {"intime_day": 1}),
    "outtime": AttrSelector("outtime", 0, {"outtime_day": 0, "outtime_week": 1}),
    "charttime": AttrSelector("charttime", 0, {"charttime_day": 0, "charttime_week": 2}),
}
m2_24 = {
    "gender": AttrSelector("gender", 0, {"gender": 0}),
    "warning": AttrSelector("warning", 0, {"warning": 0}),
    "first_careunit": AttrSelector("first_careunit", 0, {"first_careunit": 0}),
    "last_careunit": AttrSelector("last_careunit", 0, {"last_careunit": 0}),
    "intime": AttrSelector("intime", 0, {"intime_day": 0}),
    "outtime": AttrSelector("outtime", 0, {"outtime_day": 0, "outtime_week": 0}),
    "charttime": AttrSelector("charttime", 0, {"charttime_day": 0, "charttime_week": 0}),
}
m2_28 = {
    "gender": AttrSelector("gender", 0, {"gender": 0}),
    "warning": AttrSelector("warning", 0, {"warning": 0}),
    "first_careunit": AttrSelector("first_careunit", 0, {"first_careunit": 0}),
    "last_careunit": AttrSelector("last_careunit", 0, {"last_careunit": 0}),
    "intime": AttrSelector("intime", 0, {"intime_day": 0}),
    "outtime": AttrSelector("outtime", 0, {"outtime_day": 0, "outtime_week": 0}),
    "charttime": AttrSelector("charttime", 0, {"charttime_day": 0, "charttime_week": 0, "charttime_time": 1}),
}
m2_32 = {
    "gender": AttrSelector("gender", 0, {"gender": 0}),
    "warning": AttrSelector("warning", 0, {"warning": 0}),
    "first_careunit": AttrSelector("first_careunit", 0, {"first_careunit": 1}),
    "last_careunit": AttrSelector("last_careunit", 0, {"last_careunit": 0}),
    "intime": AttrSelector("intime", 0, {"intime_day": 1}),
    "outtime": AttrSelector("outtime", 0, {"outtime_day": 0, "outtime_week": 0, "outtime_time": 0}),
    "charttime": AttrSelector("charttime", 0, {"charttime_day": 0, "charttime_week": 0, "charttime_time": 0}),
}

tst = mar


def get_dom(cols):
    dom = 1
    for sel in cols.values():
        for n, h in sel.cols.items():
            dom *= new_attrs[sel.name][n].get_domain(h)
    return dom


print(f"{get_dom(tst):,}")

4,032


In [ ]:
def stub_columns(table, cols):
    return table.assign(**{c: 0 for c in cols})


table_stub = stub_columns(
    table,
    [
        "gender",
        "warning",
        "first_careunit",
        "last_careunit",
        "intime_day",
        "outtime_day",
        "outtime_week",
        "outtime_time",
        "charttime_day",
        "charttime_week",
        "charttime_time",
    ],
)


In [ ]:
import jax
import jax.numpy as jnp

def marginal_jax(ops, dom):
    out = None
    for mult, op in ops:
        if out == None:
            out = mult * op
        else:
            out += mult * op
    return jnp.zeros(dom).at[out].add(1)

marginal_jax = jax.jit(marginal_jax, static_argnums=[1])

def calc_marginal_jax(
    cols: dict[str, list[np.ndarray]],
    cols_noncommon: dict[str, list[np.ndarray]],
    domains: dict[str, list[int]],
    x: AttrSelectors,
    out: np.ndarray | None = None
):
    """Calculates the 1 way marginal of the subsections of attributes x"""

    ops = []
    mul = 1
    for attr in reversed(x.values()):
        common = attr.common
        l_mul = 1
        for i, (n, h) in enumerate(attr.cols.items()):
            if common == 0 or i == 0:
                ops.append((l_mul * mul, jnp.asarray(cols[n][h])))
            else:
                ops.append((l_mul * mul, jnp.asarray(cols_noncommon[n][h])))
            l_mul *= domains[n][h] - common
        mul *= l_mul + common

    marginal_jax(ops, mul).block_until_ready()
    %timeit marginal_jax(ops, mul).block_until_ready()

In [ ]:
from pasteur.marginal.numpy import calc_marginal_1way as calc_marginal_numpy
from pasteur.marginal.native_py import calc_marginal_1way as calc_marginal_native

def compare_marginals(x, N, desc):
    print("##############")
    print(desc)
    M = {}
    for attr in x.values():
        for n, h in attr.cols.items():
            M[n] = h
    cols, cols_noncommon, domains = expand_table(new_attrs, table[:N])

    out16 = np.zeros((get_dom(x),), dtype=np.uint32)
    dom32 = max(get_dom(x), 2**17)
    out32 = np.zeros((dom32,), dtype=np.uint32)

    print("# pandas")
    vc_df = pd.DataFrame({n: cols[n][h] for n, h in M.items()})
    %timeit vc_df.value_counts()

    print("# numpy hdd, bincount")
    hdd_np = vc_df.to_numpy()
    try:
        if get_dom(x) < 2**22:
            %timeit np.histogramdd(hdd_np, [domains[n][h] for n, h in M.items()])
    except Exception as e:
        print(e)
    %timeit calc_marginal_numpy(cols, cols_noncommon, domains, x)

    if get_dom(x) < 2**22:
        print("# jax cpu, cuda")
        with jax.default_device(jax.devices("cpu")[0]):
            calc_marginal_jax(cols, cols_noncommon, domains, x)
        with jax.default_device(jax.devices("cuda")[0]):
            calc_marginal_jax(cols, cols_noncommon, domains, x)

    print("# c, c avx 32, 16")
    %timeit calc_marginal_native(cols, cols_noncommon, domains, x, simd=False, out=out16)
    %timeit calc_marginal_native(cols, cols_noncommon, domains, x, out=out32)
    %timeit calc_marginal_native(cols, cols_noncommon, domains, x, out=out16)

    cols, cols_noncommon, domains = expand_table(new_attrs, table_stub[:N])
    print("# stalled")
    %timeit calc_marginal_native(cols, cols_noncommon, domains, x, out=out32)
    %timeit calc_marginal_native(cols, cols_noncommon, domains, x, out=out16)

In [ ]:
x = mar

for N in Ns:
    compare_marginals(x, N, f"N = {N:,}")

##############
N = 20,000
# pandas
2.39 ms ± 16.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
# numpy hdd, bincount
2.99 ms ± 9.09 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
99.2 µs ± 576 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
# jax cpu, cuda
INFO     Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker:        xla_bridge.py:355
INFO     Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm".        xla_bridge.py:355
         Available platform names are: Interpreter Host CUDA                                                                           
INFO     Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'          xla_bridge.py:355
INFO     Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client.       xla_bridge.py:355
         Compile TensorFlow wi

In [ ]:
Ms = {
    "2^8": m2_8,
    "2^12": m2_12,
    "2^16": m2_16,
    "2^20": m2_20,
    "2^24": m2_24,
    "2^28": m2_28,
    "2^32": m2_32
}

In [ ]:
for name, M in Ms.items():
    compare_marginals(M, N_dom, f"dom = {name}")

##############
dom = 2^8
# pandas
313 ms ± 755 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
# numpy hdd, bincount
449 ms ± 2.92 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
20.8 ms ± 1.18 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
# jax cpu, cuda
47.4 ms ± 286 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
2.68 ms ± 32.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
# c, c avx 32, 16
19.5 ms ± 109 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
5.11 ms ± 6.31 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.29 ms ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
# stalled
10.9 ms ± 33.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
10.8 ms ± 9.99 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
##############
dom = 2^12
# pandas
421 ms ± 7.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# numpy hdd, bincount
727 ms ± 1.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)